In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm 
from scipy import stats
%matplotlib inline

# Predicting Temperature

1. First, load the dataset from the weatherinszeged table from Thinkful's database.

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

engine.dispose()

weather_df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


1. Like in the previous checkpoint, build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why?

In [4]:
#Sort data into dependent and independent variables
Y = weather_df['apparenttemperature'] - weather_df['temperature']
X = weather_df[['humidity', 'windspeed']]
X = sm.add_constant(X)

results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Fri, 16 Aug 2019   Prob (F-statistic):               0.00
Time:                        10:47:13   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### My answer to question 1

No, I do not think the R^2 and adjusted R^2 values are satisfactory. These statistics state the regression model only explains 28% of the variance in the data, there are likely other features that should be included to better model the differences between apparent temperature and temperature.

2. Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one?

In [6]:
#Sort data into dependent and independent variables
Y = weather_df['apparenttemperature'] - weather_df['temperature']
weather_df['humidity_by_windspeed'] = weather_df['humidity'] * weather_df['windspeed']
X = weather_df[['humidity', 'windspeed', 'humidity_by_windspeed']]
X = sm.add_constant(X)

results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Fri, 16 Aug 2019   Prob (F-statistic):               0.00
Time:                        10:50:30   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.0839      0.033      2.511      0.012       0.018       0.149
humidity                  0.1775      0.043      4.133      0.000       0.093       0.262
windspeed                 0.0905      0.002     36.797      0.000       0.086       0.095
humidity_by_windspeed    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### My answer to question 2

Including the interaction of humidity and windspeed did improve the model. Both R^2 and adjusted R^2 increased from 0.288 to 0.341

3. Add visibility as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the visibility in terms of the improvement in the adjusted R-squared. Which one is more useful?

In [7]:
#Sort data into dependent and independent variables
Y = weather_df['apparenttemperature'] - weather_df['temperature']
X = weather_df[['humidity', 'windspeed', 'visibility']]
X = sm.add_constant(X)

results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Fri, 16 Aug 2019   Prob (F-statistic):               0.00
Time:                        10:53:43   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.000       1.521       1.630
humidity      -2.6066      0.025   -102.784      0.000      -2.656      -2.557
windspeed     -0.1199      0.001   -179.014      0.000      -0.121      -0.119
visibility     0.0540      0.001     46.614      0.000       0.052       0.056
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                          -0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### My answer to question 3

R^2 and adjusted R^2 both increased from the first model to the third. However, the R^2 values did not increase as much as they did in the second model. Using R^2 values exclusively to make this decision, the interaction of humidity and windspeed is more useful than visibility.

4. Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.

### My answer to question 4

Based off the AIC and BIC scores, the second model was the best as this model had the lowest score. 

# Evaluating the House prices regression model

In [9]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
prices_df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

In [10]:
#Create series of continuous variable names, check that all continuous variables are numerical

cont_vars = pd.Series(['saleprice', 'yrsold', 'miscval', 'poolarea','screenporch', 'threessnporch', 'enclosedporch',
                             'wooddecksf', 'openporchsf','garagearea', 'garageyrblt', 'grlivarea', 'lowqualfinsf', 'firstflrsf', 
                             'secondflrsf','totalbsmtsf', 'bsmtunfsf', 'bsmtfinsf1', 'bsmtfinsf2', 'masvnrarea', 
                             'yearbuilt', 'yearremodadd','lotarea', 'lotfrontage'])

#Create series of the categorical variable names

cat_vars = []

for var in prices_df.columns:
    if cont_vars.str.contains(var).any() == False:
        cat_vars.append(var)

cat_vars = pd.Series(cat_vars)

#Check that no variables are in both variable lists

for var in prices_df.columns:
    if cont_vars.str.contains(var).any() & cat_vars.str.contains(var).any():
        print(var)

In [11]:
#Convert categorical variables to numerically coded categories

coded_df = prices_df.copy()
coded_df.sort_values(by='saleprice')
codebook = {}

for var in cat_vars:
    if  coded_df[var].dtype == 'O':
        
        #Create replace dict for each variable
        labels = coded_df[var].unique()
        replace_dict = {k: v for k,v in zip(labels,range(len(labels)))}
        
        #Add var entry to codebook and replace in dataframe
        codebook.update({var: replace_dict})
        coded_df.loc[:,var] = coded_df[var].replace(replace_dict) 

In [15]:
#Boxcox transform our dependent variable, add to list of cont_vars 

coded_df['boxcox_saleprice'] = stats.boxcox(coded_df['saleprice'])[0]
cont_vars = list(cont_vars)
cont_vars.insert(0,'boxcox_saleprice')

In [16]:
#Sort variables by their relevance in the model

#Loop through cat_vars and conduct One-Way Anova across the groups in each cat_var for boxcox_saleprice

import statsmodels.api as sm
from statsmodels.formula.api import ols
F = []
p = []

for var in cat_vars:
    mod_str = 'boxcox_saleprice ~ ' + var
    mod = ols(mod_str, data=coded_df).fit()
    aov_table= sm.stats.anova_lm(mod, typ=2)
    F.append(aov_table['F'][0])
    p.append(aov_table['PR(>F)'][0])
    
#Save results of anova in df with variable names

aov_res = pd.DataFrame(cat_vars, columns=['var'])
aov_res['F'] = F
aov_res['p-value'] = p 
aov_res.sort_values(by='F', ascending=False, inplace=True)

#Create correlation matrix, sort by saleprice
cont_corrs = coded_df[cont_vars].corr()
cont_corrs.sort_values(by='boxcox_saleprice',axis=0, ascending=False, inplace=True)

In [23]:
#Create series of top 10 categorical variables
features = aov_res['var'][:10]

#Append series of continuous variables with greater than 0.5 correlation and remove saleprice vars and secondflrsf
cont_features = list(cont_corrs.loc[cont_corrs['boxcox_saleprice']>0.5, 'boxcox_saleprice'].index)
for var in ['boxcox_saleprice', 'saleprice']: cont_features.remove(var) 
    
features = features.append(pd.Series(cont_features), ignore_index=True)

In [18]:
coded_df.loc[:,features] = coded_df.loc[:,features].fillna(0)

In [19]:
#Sort data into dependent and independent variables
Y = coded_df['boxcox_saleprice']
X = coded_df[features]
X = sm.add_constant(X)


results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       boxcox_saleprice   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     422.2
Date:                Fri, 16 Aug 2019   Prob (F-statistic):               0.00
Time:                        11:09:01   Log-Likelihood:                 1925.6
No. Observations:                1460   AIC:                            -3815.
Df Residuals:                    1442   BIC:                            -3720.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            4.6515      0.271     17.188      0.000       4.121       5.182
overallqual      0.0319      0.002     15.102      0.000       0.028       0.036
garagecars       0.0174      0.006      3.147      0.002       0.007       0.028
fullbath        -0.0020      0.005     -0.427      0.669      -0.011       0.007
totrmsabvgrd     0.0052      0.002      2.677      0.008       0.001       0.009
fireplaces       0.0275      0.004      6.561      0.000       0.019       0.036
foundation      -0.0069      0.003     -2.324      0.020      -0.013      -0.001
heatingqc       -0.0087      0.002     -3.910      0.000      -0.013      -0.004
fireplacequ      0.0016      0.002      0.688      0.492      -0.003       0.006
bsmtfintype1    -0.0031      0.001     -2.556      0.011      -0.005      -0.001
exterqual       -0.0029      0.004     -0.835      0.404      -0.010       0.004
grlivarea     6.735e-05   7.51e-06      8.967      0.000    5.26e-05    8.21e-05
garagearea    3.075e-05    1.8e-05      1.706      0.088    -4.6e-06    6.61e-05
totalbsmtsf   2.788e-05    7.9e-06      3.530      0.000    1.24e-05    4.34e-05
firstflrsf    2.336e-05      9e-06      2.594      0.010     5.7e-06     4.1e-05
yearbuilt        0.0006   9.57e-05      6.136      0.000       0.000       0.001
yearremodadd     0.0008      0.000      6.664      0.000       0.001       0.001
garageyrblt   1.774e-05   4.93e-06      3.598      0.000    8.07e-06    2.74e-05
==============================================================================
Omnibus:                      952.949   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34266.485
Skew:                          -2.501   Prob(JB):                         0.00
Kurtosis:                      26.201   Cond. No.                     6.46e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.46e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

1. Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
2. Do you think your model is satisfactory? If so, why?

### My answer to question 1 and 2

The model is good based on these metrics, the model explains ~83% of the variance in House Prices. The F-Test is significant. AIC and BIC values are very low, but without a similar model to compare to it is difficult to say if the model is "good" or not using this metric. I would not yet call the model satisfactory as there are still issues with multicollinearity and heteroscedasticity that have not been adressed. Additionally, the R^2 values could probably be improved with further modification of the features. 

3. In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables.
4. For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In [24]:
#Sort data into dependent and independent variables
Y = coded_df['boxcox_saleprice']
features = list(features)
for var in ['fullbath', 'fireplacequ', 'exterqual', 'garagearea']: features.remove(var)
X = coded_df[features]
X = sm.add_constant(X)


results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       boxcox_saleprice   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     551.6
Date:                Fri, 16 Aug 2019   Prob (F-statistic):               0.00
Time:                        11:09:59   Log-Likelihood:                 1923.4
No. Observations:                1460   AIC:                            -3819.
Df Residuals:                    1446   BIC:                            -3745.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            4.6407      0.257     18.047      0.000       4.136       5.145
overallqual      0.0322      0.002     15.349      0.000       0.028       0.036
garagecars       0.0241      0.004      6.356      0.000       0.017       0.032
totrmsabvgrd     0.0047      0.002      2.481      0.013       0.001       0.008
fireplaces       0.0289      0.003      9.270      0.000       0.023       0.035
foundation      -0.0070      0.003     -2.349      0.019      -0.013      -0.001
heatingqc       -0.0090      0.002     -4.062      0.000      -0.013      -0.005
bsmtfintype1    -0.0031      0.001     -2.605      0.009      -0.005      -0.001
grlivarea     6.794e-05   7.13e-06      9.533      0.000     5.4e-05    8.19e-05
totalbsmtsf   2.899e-05   7.83e-06      3.703      0.000    1.36e-05    4.44e-05
firstflrsf    2.423e-05   8.96e-06      2.704      0.007    6.65e-06    4.18e-05
yearbuilt        0.0006   9.12e-05      6.405      0.000       0.000       0.001
yearremodadd     0.0008      0.000      6.770      0.000       0.001       0.001
garageyrblt   1.933e-05   4.85e-06      3.986      0.000    9.82e-06    2.88e-05
==============================================================================
Omnibus:                      934.329   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            31464.688
Skew:                          -2.450   Prob(JB):                         0.00
Kurtosis:                      25.209   Cond. No.                     6.09e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.09e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### My answer to question 3 and 4

While removing the insignificant features did not impprove the R^2 values, it did decrease the AIC and BIC values. This indicates that the second model is a better choice than the first.